# Real-Time Dynamics in a (2+1)-D Gauge Theory
#### **Authors:** Jesus Cobos, Joana Fraxanet
#### **Reference link:** [Real-Time Dynamics in a (2+1)-D Gauge Theory: The Stringy Nature on a Superconducting Quantum Simulator](https://arxiv.org/pdf/2507.08088)

In this challenge, we will simulate the **time evolution of the $\mathbb{Z}_2$-Higgs model** using a **second-order Trotter decomposition**.  
This method allows us to approximate the dynamics by breaking the time evolution operator into smaller, more manageable steps.  

---

### Hamiltonian structure
The Hamiltonian describes the dynamics of both **matter fields** and **gauge fields**, which are represented by different sets of qubits in the lattice:

<img src="images_LAB1/heavyhex.png" width="400"/>

- **Matter fields ($\tau$ operators):** live on the **sites** of the lattice indicated with circles.  
- **Gauge fields ($\sigma$ operators):** live on the **links** connecting neighboring matter fields (indicated with squares).   

Both $\tau$ and $\sigma$ here correspond to [Pauli Z operators](https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.quantum_info.Pauli). Following the standard formulation:

- The **local terms** are  
  $$
  H_M = - J \sum_{n\in \bigcirc} \tau^z_{n}
  - h \sum_{v\in\square} \sigma^z_{v},
  $$
  where the first sum corresponds to the on-site energy of the matter qubits and the second to the energy of the gauge-field link qubits connecting two matter sites.  

- The **interaction term** is  
  $$
  H_I = - \lambda \sum_{\text{links } \ell=(n,m)} 
    \tau^x_{n} \, \sigma^x_{\ell} \, \tau^x_{m},
  $$
  which couples matter and gauge qubits in a gauge-invariant way. These connections happen between matter qubits and are mediated with the gauge-field qubits that exist in between them.

The $\mathbb{Z}_2$ gauge symmetry is generated by $G_n=\tau^z_n \prod_{v\in\ell_n}\sigma^z_{(n,v)}$, which commutes with $H$. Physical states fulfill the Gauss’ law constraint $G_n|\psi\rangle=|\psi\rangle$.

---

### Circuit interpretation

A single Trotter step for time step $dt$ is given by:

$$
U_T(t) = e^{-i H_M dt/2} \, e^{-i H_I dt} \, e^{-i H_M dt/2},
$$

- The exponential of $H_M$ reduces to **single-qubit rotations** acting separately on site and link qubits.  
- The exponential of $H_I$ requires **two-qubit entangling gates**. Each site is connected to several links, so the number of such gates per Trotter step is $2 \times \texttt{connectivity}$, where the connectivity is maximum number of gauge links attached to each matter site (which equals $3$ for the heavy-hex lattice).  

By repeating these Trotter steps, we build up the full time evolution of the system. The reachable simulation time depends on two competing factors:  

- **Timestep $dt$:** smaller $dt$ reduces Trotter error, but increases the number of circuit depth needed.  
- **Circuit depth:** larger depth increases hardware error, which eventually limits the simulation fidelity.  

Finding the right balance between these sources of error, while reducing hardware noise as much as possible through **error suppression, mitigation, and correction strategies** is crucial for pushing the simulation to utility scales.

<a id="step1"></a>
<div class="alert alert-block alert-success">
<img align="right" width="320" src="images_LAB1/classicaldynamics.png" >

**Your task:**  

We provide a specific point in the phase diagram $(J, h, \lambda)$ along with the corresponding classical dynamics for reference (see image on the side).  Your goal is to implement the **quantum dynamics** and simulate the evolution of the system with the **highest possible accuracy** with regard to the classical benchmark, which is obtained using state-of-the-art tensor networks.

**Key details:**

<img src="images_LAB1/initial_state.png" width="450"/>

- **Lattice:** $2 \times 2$ plaquettes.  
- **Initial state:** a vertical string connecting the endpoint qubits (see figure above).  
- **Parameters:** we choose the point $J=5, h=2, \lambda=1$.
- **Total time:** simulate the evolution from $t=0$ to $t=3$.

This task will be graded out of 100 points, distributed as follows: A – 5 points, B – 10 points, C – 20 points, D – 5 points, and E – 60 points.

</div> 

### **Step 1.** Choosing your device and initialization

If this is your first challenge, follow the steps in the `INSTALL.md` file to set up your environment for all challenges:

- Create an IBM Cloud account and make sure to save your account credentials as indicated.
- Clone the repository locally if you haven’t already.
- Create and activate a Python virtual environment.
- Install the required packages listed in the `requirements.txt` — this includes the graders.

Once the setup is ready, you can import the graders corresponding to this notebook and set your team name:

In [ ]:
import qc_grader
from qc_grader.challenges.qdc_2025 import submit_name, grade_lab1_ex1, grade_lab1_ex2, grade_lab1_ex3, grade_lab1_ex4, grade_lab1_ex5

team_name = #Add string
submit_name(team_name)

Now you are ready to start initializing the variables for your task!
Read the challenge instructions above and fill in the right numbers:

In [ ]:
######### EXERCISE #########

from qiskit_ibm_runtime import QiskitRuntimeService
import pandas
import numpy as np

# Device 
service = QiskitRuntimeService(name = 'qdc-2025')
device_name = #Add string with device name
backend = service.backend(device_name)

# Lattice
plaquette_width = #Number of horizontal plaquettes of "Key details" image
plaquette_height = #Number of vertical plaquettes of "Key details" image

# Parameters
J = #Add int
h = #Add int
lamb = #Add int

# Classical dynamics
classical_dynamics = pandas.read_csv('classical_benchmark.txt', sep=' ', header=None)
classical_exp_vals = np.array(classical_dynamics).T[:-1]

### **Step 2.** Prepare the lattice and the quantum circuit for the initial state

Here we will start constructing Quantum Circuits.

Start by importing the lattice creation and visualization class from the file `lattice.py`.

In [ ]:
from lattice_utils import *

lattice = HeavyHexLattice(plaquette_width, plaquette_height)
lattice.plot_lattice(number_qubits=True)

# HINTS TO USE THE LATTICE PACKAGE:

# Total number of qubits
print('Total number of qubits:', len(lattice.coords))

# Accessing Gauge-field link qubit indices (square)
print([link.qubit for link in lattice.edges.values()])
# Accessing matter qubit indices (circle)
print([node.qubit for node in lattice.vertices.values()])

In this first exercise, you need to prepare the initial state by flipping (i.e., setting to the excited state) the qubits that represent **the matter sites at the ends of the string and the link qubits along the path that connects them**.

In [ ]:
######### EXERCISE #########

from qiskit.circuit import QuantumCircuit

def StringInitialStateCirc(num_qubits, string_qubits):
    """
    Build a circuit that prepares the initial state (following figure in "Key details" image)

    Args:
        num_qubits (int): Total number of qubits of the system (all plaquettes)
        string_qubits (list[int]): Indices of qubits to flip.

    Returns:
        QuantumCircuit: A quantum circuit object.
    """

    # Add code here


num_qubits = #Add
string_qubits = #Add
init_state = StringInitialStateCirc(num_qubits, string_qubits)
lattice.plot_highlighted_qubits(string_qubits, colors=["#2f64f7"]*len(string_qubits), number_qubits=True)

<div class="alert alert-block alert-success">

**Grader A [5 points]:**  

Let's check if your initial circuit reproduces the right result.

</div> 

In [ ]:
qc = StringInitialStateCirc(num_qubits, string_qubits)

# Submit your answer using following code
grade_lab1_ex1(qc)

### **Step 3.** Implement the time evolution operators

Here we will start preparing the circuits to simulate the evolution under the Z2 LGT Hamiltonian.

#### $H_M$ Hamiltonian

Here we focus on the single-qubit part of the Hamiltonian.

$$
H_M = - J \sum_{n\in \bigcirc} \tau^z_{n} - h \sum_{v\in\square} \sigma^z_{v}
$$

The goal is to implement the time evolution operator:

$$
e^{-i H_M \, dt}
$$

> **Note:** In Qiskit, [Pauli rotations](https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.circuit.library.RZGate) use half-angles $R_P(θ)=e^{−iθ/2·P}$, so the evolution operator $e^{−iA·P·dt}$ is implemented as $R_P(2·A·dt)$.


In [ ]:
######### EXERCISE #########

def SingleQubitEvolution(lattice, J, h, dt):

    """
    Constructs a quantum circuit that implements the time evolution of the 
    single-qubit part of the Hamiltonian, using a time dt.

    Args:
        lattice (object): Lattice object defined above
        J (float): Single-qubit Hamiltonian parameter
        h (float): Single-qubit Hamiltonian parameter
        dt (float): time step for trotter circuits

    Returns:
        QuantumCircuit: A quantum circuit object acting on all qubits of the system.
    """

    # Add code here


<div class="alert alert-block alert-success">

**Grader B [10 points]:**  

Let's check if your circuit implements the single-qubit evolution operator correctly.

</div> 

In [ ]:
qc = SingleQubitEvolution(lattice, J, h, 1) #Single-qubit time evolution circuit with Hamiltonian parameters defined in "key details" and dt = 1, J=5, h=2

# Submit your answer using following code
grade_lab1_ex2(qc, lattice)

#### $H_I$ Hamiltonian

Now, we will focus on the interacting part of the Hamiltonian.

$$
H_I = - \lambda \sum_{\text{links } \ell=(n,m)} \tau^x_{n} \, \sigma^x_{\ell} \, \tau^x_{m}
$$

The goal is to implement the corresponding time evolution operator:

$$
e^{-i H_I \, dt}
$$

This requires entangling gates, and careful scheduling is essential to minimize circuit depth.

> **Note:** In Qiskit, [Pauli rotations](https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.circuit.library.RXGate) use half-angles $R_P(θ)=e^{−iθ/2·P}$, so the evolution operator $e^{−iA·P·dt}$ is implemented as $R_P(2·A·dt)$.

##### Instructions for implementation

The scheduling of entangling gates in this implementation is crucial, since we want to minimize the overall circuit depth. Our goal is to compress all gates into as few layers as possible by adopting an effective circuit construction strategy. 

This strategy is outlined below. The idea is to use the `lattice` object to access qubit coordinates and build the Trotter circuits using a clever scheduling.
- **Pros**: Allows for **depth optimization** needed to obtain accurate results.
- **Cons**: Requires careful scheduling and lattice-aware design.

_______________________

If this section is too complicated for you, there is an easy fix to proceed to the end without getting points in Grader C. You can represent $H_I$ as a [SparsePauliOp](https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.quantum_info.SparsePauliOp) and use Qiskit’s built-in evolution tools.
- **Pros**: Straightforward to implement.
- **Cons**: Results in circuits with **high depth**, which will reduce the accuracy of the results due to noise and limited coherence time.

> **Note:** If you implement both strategies, it might be insightful to compare the number of two-qubit gate layers using `circuit.depth(lambda x: len(x.qubits) == 2)`.

_______________________

<div class="alert alert-block"> 

#### Instructions for strategy 1: Manual circuit construction

Start by splitting the lattice into two sets of matter qubits (circles):

- 🟩 Green
- 🟪 Purple

as indicated in the figure:

<img src="images_LAB1/schedulecx.png" width="700"/>

where each matter qubit is controlled by a different gauge-field qubit at each step (arrows here indicate the changes layer to layer). Each block C is built in three layers, following this pattern:

**Layer 1:**

- Green qubits connect to the qubit above.
- Purple qubits connect to the qubit on the left.

**Layer 2:**

- Both purple and green qubits connect to the green qubit on their right.

**Layer 3:**

- Green qubits connect to the qubit on the left.
- Purple qubits connect to the qubit below.

All these connections are implemented using `cx` gates, where the control qubit is always the gauge-field qubit (i.e., the one on the edge).

Then, the function preparing a Trotter step will include:

- One block `C`
- `Rx` rotations on gauge link qubits
- The inverse of block `C`, as shown in the diagram.

\
<img src="images_LAB1/interacting_circ.png" width="250"/>

To implement this efficiently, we recommend using the following features of the lattice package:

In [ ]:
# HINTS TO USE THE LATTICE PACKAGE:

# Accessing qubit coordinates, which are ordered as qubit indices in Step 1.
# The distance between matter qubits is 1, while the distance between matter and field gauge qubits is 0.5.
print('Gauge-field qubit coordinates:',[link.coords for link in lattice.edges.values()])
print('Matter qubit coordinates:',[node.coords for node in lattice.vertices.values()])
 
# Split matter qubits into two sets
matter_qubits_green = lattice.find_qubits_downward()
matter_qubits_purple = lattice.find_qubits_upward()
lattice.plot_highlighted_qubits(matter_qubits_green, colors=["#1ebb36"]*len(matter_qubits_green), number_qubits=True)
lattice.plot_highlighted_qubits(matter_qubits_purple, colors=["#af23de"]*len(matter_qubits_purple), number_qubits=True)

# Accessing a qubit by its coordinate. 
# The placements are NOT STRAIGHTFORWARD. They follow an axis with origin on the top left of the lattice.
# The following examples should help:

# Locate matter qubit
coordinate = (0, 3)
index = lattice.vertices[coordinate].qubit
print(f'Matter qubit corresponding to coordinate {coordinate} has index {index}.')

# Gauge link on the left
new_coordinate = (coordinate[0], coordinate[1]-0.5) 
index = lattice.edges[new_coordinate].qubit
print(f'Gauge link qubit corresponding to coordinate {new_coordinate} has index {index}.')

# Gauge link on the right
new_coordinate = (coordinate[0], coordinate[1]+0.5)
index = lattice.edges[new_coordinate].qubit
print(f'Gauge link qubit corresponding to coordinate {new_coordinate} has index {index}.')

# Gauge link below
new_coordinate = (coordinate[0]+0.5, coordinate[1])
index = lattice.edges[new_coordinate].qubit
print(f'Gauge link qubit corresponding to coordinate {new_coordinate} has index {index}.')

In [ ]:
######### EXERCISE #########

def InteractionEvolution(lattice, lamb, dt):

    """
    Constructs a quantum circuit that implements the time evolution of the 
    interacting part of the Hamiltonian, using a time dt.

    Args:
        lattice (object): Lattice object defined above
        lamb (float): interaction Hamiltonian parameter
        dt (float): time step for trotter circuits

    Returns:
        QuantumCircuit: A quantum circuit object acting on all qubits of the system.
    """

    # Add code here


<div class="alert alert-block alert-warning"> 

**Optional Bonus: Gauss Dynamical Decoupling (GDD) for Error Suppression**

This technique is designed to protect simulations from noise-induced transitions into non-physical states—i.e., states that violate the Gauss’ law constraint.

**Procedure:**

1. **Random Phase Generation:** For each Trotter step $k$ and matter site $n$, draw a random phase $\phi_{k,n}$ uniformly from $[-π, π]$.
2. **Gauge Operator Insertion:** Apply these phases via the gauge operators $G_n$ within the Trotterized evolution, resulting in an effective evolution:

$$
\widetilde{U}(t) = \prod_k \bigl[ U_1(dt/2)\, U_3(dt)\, U_G(\{\varphi_{k,n}\})\, U_1(dt/2) \bigr],\ \ \ \ U_G(\{\phi_{k,n}\}) = \prod_n e^{-i \phi_{k,n} G_n},
$$

> **Note:** An important constraint is that the sum of phases for all Trotter steps at each site is zero.

In practice, the new circuits will look like:

<img src="images_LAB1/interacting_circ_GDD.png" width="250">


**Effect:**

* Since the Hamiltonian commutes with all $G_n$, this procedure does not alter the ideal evolution of physical states.
* Any non-physical components (off-diagonal terms in the unphysical subspace) acquire random phases, effectively suppressing noise-driven leakage out of the physical sector.

</div> 

<div class="alert alert-block alert-success">

**Grader C [20 points]:**  

Let's check if your circuit implements the interaction evolution operator correctly.

</div> 

In [ ]:
qc = InteractionEvolution(lattice, lamb, 1) ##Interaction time evolution circuit with Hamiltonian parameters defined in "key details" and dt = 1, lamb=1

# Submit your answer using following code
grade_lab1_ex3(qc, lattice)

### **Step 4.** Implement the full evolution circuit 
 

Here, we will start by building a function that constructs a single second-order Trotter step by composing the single-qubit and interaction quantum circuits you created above.
Recall that:

$$
U_T(t) = e^{-i H_M dt/2} \, e^{-i H_I dt} \, e^{-i H_M dt/2}
$$

In [ ]:
######### EXERCISE #########

def SecondOrderTrotter(lattice, J, h, lamb, dt):
        
    """
    Constructs a quantum circuit that performs a single second order
    Trotter step of the full Hamiltonian, using a time increment of dt.

    Args:
        lattice (object): Lattice object
        J (float): Hamiltonian parameter
        h (float): Hamiltonian parameter
        lamb (float): Hamiltonian parameter
        dt (float): time step for trotter circuits

    Returns:
        QuantumCircuit: A quantum circuit object acting on all qubits of the system.
    """

    # Add code here

**Now we are ready to define a function that constructs the complete set of quantum circuits required to simulate time evolution across discrete time steps!**

- Begin by initializing a circuit that prepares the initial quantum state.
- Sequentially append one Trotterized evolution layer per time increment.
- For each appended layer, output the corresponding circuit to enable tracking of the system's evolution over time.

Refer to Fig. 4 of the [paper](https://arxiv.org/pdf/2507.08088) to identify the optimal balance between the Trotter time step $dt$ and the overall circuit depth.
The objective is to select a $dt$ that is sufficiently large to keep circuit depth manageable, yet small enough to maintain the fidelity of the simulation despite Trotter error.

In [ ]:
######### EXERCISE #########

def string_quench_simulation_circuits(lattice, J, h, lamb, string_qubits, dt, total_layers):

    """
    Constructs a set of quantum circuit required to 
    simulate time evolution across discrete time steps.

    Args:
        lattice (object): Lattice object
        J (float): Hamiltonian parameter
        h (float): Hamiltonian parameter
        lamb (float): Hamiltonian parameter
        string_qubits (list): indices of qubits to flip in initial state
        dt (float): time step for trotter circuits
        total_layers (int): maximum number of Trotter steps 

    Returns:
        list[QuantumCircuit]: A list of quantum circuit objects (one per time step)
    """

    # Add code here



In [ ]:
final_time = 3
dt = #Add
total_layers = int(final_time/dt)
logical_circuits = string_quench_simulation_circuits(lattice, J, h, lamb, string_qubits, dt, total_layers)

In [ ]:
time_step_circuit = 1 #for example
logical_circuits[time_step_circuit].draw("mpl", idle_wires=False, scale=0.5, fold=True)

### **Step 5.** Transpile the circuits
 

Transpile the quantum circuits to the backend using a `generate_preset_pass_manager`. 

To fix the same layout to all the circuits, choose an initial layout based on the shallowest circuit (using all qubits) and apply it to the rest. For example:

```python
init_layout = shallow_isa_circuit.layout
pass_manager = generate_preset_pass_manager(
    optimization_level=3,
    backend=backend,
    initial_layout=init_layout.final_index_layout(),
)
```

> **Note:** If you want transpilation to be more efficient, you can transpile per circuit slice (all Trotter steps have the same structure).

In [ ]:
######### EXERCISE #########

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.visualization import plot_circuit_layout # Note: graphviz might be needed.

def transpile_circuits(logical_circuits, optimization_level, backend):

    """
    Use generate_preset_pass_manager to transpile all logical 
    circuits to ISA circuits with the same optimized layout.

    Args:
        logical circuits (list[QuantumCircuit]): list of quantum circuits
        optimization_level (int): 1, 2 or 3
        backend (Qiskit backend object): backend object

    Returns:
        list[QuantumCircuit]: A list of ISA quantum circuit objects per each time step
    """

    # Add code here
    
    
optimization_level = 3
isa_circuits = transpile_circuits(logical_circuits, optimization_level, backend)

largest_circuit = isa_circuits[-1]
print(f'2Q depth of largest circuit is: {largest_circuit.depth(lambda x: len(x.qubits) == 2)}')
print(f'Num. of gates of largest circuit is: {largest_circuit.count_ops()}')
plot_circuit_layout(largest_circuit, backend, view = "virtual") # If you don't manage to install GraphViz, comment this line and proceed

<div class="alert alert-block alert-success">

**Grader D [5 points]:**

Let's verify whether the ISA circuits you're obtaining have reasonable volumes in terms of qubit count and circuit depth.

</div>

In [ ]:
# Submit your answer using following code
grade_lab1_ex4(largest_circuit, lattice, dt, backend)

### **Step 6.** Prepare the set of observables to measure

In this notebook, we aim to measure qubit occupancies in the $Z$ basis. 

To this end, we construct a set of observables, where each observable applies a $Z$ operator to every qubit in the plaquette system. We then generate a list of mapped observables, with each entry corresponding to the collection of `num_qubit` observables adapted to the transpiled layout of the associated `isa_circuit`.

> **Note:** In Qiskit, observables are ordered using little-endian notation, which is reversed compared to the notation for circuits. 

In [ ]:
######### EXERCISE #########

from qiskit.quantum_info import SparsePauliOp

observables = SparsePauliOp(["I"*qubit_ind + "Z" + "I"*(num_qubits - qubit_ind - 1) for qubit_ind in range(num_qubits)])

def transpile_observables(isa_circuits, observables):

    """
    Transpile all logical observables to 
    ISA observables for each ISA circuit.

    Args:
        isa_circuits (list[QuantumCircuit]): list of ISA circuits
        observables (list[SparsePauliOp]): list of observables per ISA circuit

    Returns:
        list[list[SparsePauliOp]]: A list of ISA observables list per ISA quantum circuit 
    """

    # Add code here

isa_observables = transpile_observables(isa_circuits, observables)
print(isa_observables)

### **Step 7.** Execute using the EstimatorV2 primitive

For each `pub` containing a `isa_circuit` and its corresponding set of `isa_mapped_observables`, use **EstimatorV2** to submit a job to the backend you selected initially. Store the resulting job objects so they can be retrieved and printed later. Make use of the [Batch execution mode](https://quantum.cloud.ibm.com/docs/en/api/qiskit-ibm-runtime/batch) to efficiently run multiple circuits together.

Experiment with different configurations of [error suppression and mitigation techniques available in Qiskit Runtime](https://quantum.cloud.ibm.com/docs/en/guides/error-mitigation-and-suppression-techniques). You can specify these, along with the number of shots, through the `estimator_options`.

> **Note:** As a reference, running approximately 20,000 shots with 3 ZNE factors and 20 time steps takes just under 10 minutes in Batch mode. Make sure your total time stays below the 10-minute limit; otherwise, the batch will not execute entirely.

<div class="alert alert-block alert-danger">
Be sure to verify all settings and parameters before running on hardware, as you will have limited execution opportunities.
</div> 

<div class="alert alert-block alert-info">

**For advanced users:**

If you’d like to explore a more sophisticated method to further refine your results, you’ll find detailed instructions at the end on how to run your simulations using the Sampler and apply postselection/Gauss sector correction, following the procedure outlined in the [paper](https://arxiv.org/pdf/2507.08088).

You may skip the EstimatorV2 step and attempt the bonus section instead for a chance to achieve a higher grade in the final exercise.

Please note that this advanced part is considerably more complex and may require additional time to implement.

</div> 

In [ ]:
########## EXERCISE #########

from qiskit_ibm_runtime import Batch, EstimatorV2

def execute_estimator_batch(backend, estimator_opt_dict, isa_circuits, mapped_observables):    

    """
    Use batch to run jobs in hardware with EstimatorV2.

    Args:
        backend (Qiskit backend object): backend object
        estimator_options (dict): estimator options dictionary
        isa_circuits (list[QuantumCircuit]): list of ISA circuits
        isa_observables (list[SparsePauliOp]): list of ISA observables per ISA circuit

    Returns: List of job objects 
    """
    
    job_objs = []

    with Batch(backend=backend, max_time="10m") as batch:

        print(f'Session ID on {backend}: ', batch.session_id)
        
        # Add code here

    return job_objs 

In [ ]:
########## EXERCISE #########

shots = #choose something between 10000 and 25000 (be aware of the QPU time needed!)

estimator_options = {
    "default_shots": shots,
    "resilience_level": 0,
        "resilience": {
            "zne_mitigation": True, 
            "measure_mitigation": True,
            "pec_mitigation": False, 
            "zne": {
                "extrapolator": ("exponential", "linear"),
                "amplifier": "gate_folding", 
                "noise_factors":  #choose 3 noise factors between 1 and 2
            }
        },
    "dynamical_decoupling": {
        "enable": True,
        "sequence_type": #choose the sequence that you prefer (see docs for dynamical decoupling)
    },
    "twirling": {
        "enable_gates": True,
        "enable_measure": True,
        "num_randomizations": "auto",
        "shots_per_randomization": "auto",
    }
}

# Uncomment below when ready:
#job_objs = execute_estimator_batch(backend, estimator_options, isa_circuits, isa_observables)
#print(job_objs)

### **Step 8.** Plot results from EstimatorV2 and compare with classical dynamics

Once all the jobs in the Batch Session have been executed (check the status in the [quantum platform](https://quantum.cloud.ibm.com/)), you can retrieve the jobs using the following code:

In [ ]:
########## EXERCISE #########

session_id = #Add
jobs = service.jobs(limit=100, session_id=session_id)
job_ids = [j.job_id() for j in jobs]

Now we can post-process the results and get them ready for plotting.

> **Note:** If some jobs failed because the batch exceeded the total time allowed, do not worry. You will only be graded on the accuracy of the points you obtained.

In [ ]:
expectation_vals_zne = []
standard_errors_zne = []
expectation_vals_raw = []
standard_errors_raw = []

for i, job_id in enumerate(job_ids[::-1]):
    
    try:

        job = service.job(job_id)
        pub_result = job.result()[0]

        expectation_vals_raw.append(pub_result.data.evs_noise_factors[:, 0])
        standard_errors_raw.append(pub_result.data.stds_noise_factors[:,0]) 

        expectation_vals_zne.append(pub_result.data.evs)
        standard_errors_zne.append(pub_result.data.stds) 

    except:

        print(f'Job {i} did not run.')

expectation_vals_raw = np.array(expectation_vals_raw).T
standard_errors_raw = np.array(standard_errors_raw).T

expectation_vals_zne = np.array(expectation_vals_zne).T
standard_errors_zne = np.array(standard_errors_zne).T

Finally, we can plot the evolution of the system alongside the classical benchmark for different qubits. 
In this example, we check the qubits highlighted in the picture:

<img src="images_LAB1/num_qubits.png" width="500"/>

> **Note:** You can adjust the qubit indices to explore your results—later, since you will need to select the best-performing qubit for evaluation. 
Refer to the lattice diagram above to identify the index of the qubit under study.

In [ ]:
import matplotlib.pyplot as plt
plt.rc("text", usetex=True)
plt.rc("font", size=20, family="serif", weight="bold")

for qubit in [4, 30, 14, 24]: 

    plt.figure(figsize=[9, 6])

    time_steps = np.arange(1, len(expectation_vals_zne[qubit])+1)*0.15

    # Without ZNE RAW 
    y = (1-expectation_vals_raw[qubit])/2
    plt.plot(time_steps, y, "o", markersize=8, markeredgecolor="black", color='tab:blue', label=f'RAW qubit {qubit}')
    plt.fill_between(time_steps, y - standard_errors_raw[qubit], y + standard_errors_raw[qubit], alpha=0.3, color='tab:blue')

    # With ZNE 
    y = (1-expectation_vals_zne[qubit])/2
    plt.plot(time_steps, y, "o", markersize=8, markeredgecolor="black", color='tab:green', label=f'ZNE qubit {qubit}')
    plt.fill_between(time_steps, y - standard_errors_zne[qubit], y + standard_errors_zne[qubit], alpha=0.3, color='tab:green')

    # Classical benchmark
    time_steps = np.arange(1, len(classical_exp_vals[qubit])+1)*0.01
    y = (1-classical_exp_vals[qubit])/2
    plt.plot(time_steps, y, "-", color='grey', label=f'Classical benchmark')

    plt.legend(loc="best", fontsize=16)
    plt.grid(color="grey", linewidth=1, zorder=0, alpha=0.2)
    plt.xlabel(r'$\lambda t$')
    plt.ylabel(rf'$(1-<\tau_nm^z>)/2$')
    plt.title(f'Estimator {plaquette_width}x{plaquette_height}, m = {J}, g={h}, dt={dt}, shots={shots}')
    plt.ylim(0,1.3)
    plt.xlim(0,5)

<div class="alert alert-block alert-success">

**Grader E [60 points]:**

Now it’s time to choose your best result — whether it’s postselected, ZNE-corrected, or simply the raw Estimator output — for the qubit that performed best in your plots. This selected result will be used for the final grading step.

Run the evaluation function below to compare your quantum simulation with the classical benchmark. The closer the distance is to zero, the smaller the difference between your evolution and the classical reference. Let’s see how well you’ve done! 

Here's the scoring based on the distance value:

- **distance < 0.1:** 60/60 points
- **0.1 < distance < 0.3:** 55/60 points
- **0.3 < distance < 0.5:** 50/60 points
- **0.5 < distance < 1:** 40/60 points
- **1 < distance < 1.5:** 30/60 points
- **1.5 < distance < 2:** 20/60 points
- **2 < distance < 5:** 10/60 points
- **distance > 5:** 5/60 points

Please make sure your results are genuine — automated checks are in place to ensure fairness. Submissions found to be invalid will receive 0/60 points.

Finally, we’ll review the notebooks from the top-performing teams to highlight and celebrate the best results!

</div>

In [ ]:
best_expectation_vals = #Add the full matrix of results for all observables (do not use the classical benchmark here or you will be graded with 0 points)
qubit = #Add the qubit index you have chosen
dt = #Add the dt you have used to obtain the results (needed for evaluation)

# Submit your answer using following code
grade_lab1_ex5(best_expectation_vals, qubit, dt, classical_exp_vals)

### You’ve reached the end of the notebook—congratulations on making it this far! 🎉

_____________________________

<div class="alert alert-block alert-info">

### **Optional bonus:** Execute circuits using the SamplerV2 primitive  
#### Implementing Postselection  

As an alternative approach, you can run your jobs using the **SamplerV2** primitive to obtain **raw bitstrings**.  
Unlike the EstimatorV2, this method does not directly evaluate a list of observables — instead, the observables are reconstructed afterward from the measured data.  

> If you choose this option, start by executing the `isa_circuits` using **SamplerV2**.  
> You can easily adapt the **EstimatorV2** code provided above for this purpose.  
> In this case, the number of shots can be increased to **10,000**, with **3 ZNE factors** and **20 time steps**, which typically takes just under **10 minutes** in Batch mode.  
> Ensure your total runtime remains **below the 10-minute limit**, or the batch will not execute.  

When implementing **postselection**, your goal is to analyze the raw bitstrings obtained from the QPU and **reconstruct on-site occupancies** after postselecting only those bitstrings that satisfy the **Gauss law**, discarding all others.  

Follow this workflow:  

1. **Define the relevant operators**  

   * Construct local $Z$-Pauli operators (one per qubit).  
   * Define the *Gauss law (gauge) operators* based on the lattice:  
     $G_n = \tau^z_n \prod_{v \in \ell_n} \sigma^z_{(n,v)}$  
   * Collect these into sets of observables and postselection operators.  

2. **Postselection**  

   * Use the Gauss law operators to filter out measurement outcomes (bitstrings) that are **not physically valid**:  
      $G_n |\psi\rangle \neq |\psi\rangle$.
   * This ensures that only samples consistent with the **gauge constraints** are retained.  

3. **Reconstruct observables from bitstrings**  

   * From the valid samples, compute expectation values of **diagonal observables**.  
   * Carefully match the order of observables to the qubit layout in your logical circuits so that indices remain consistent with previous steps.  
   * Compare results **with and without postselection** to assess performance improvements.  

You are free to organize your code as you prefer — for instance, you might create helper functions for postselection, observable measurement, and related tasks.  

In the [paper](https://arxiv.org/abs/2507.08088), we take this approach a step further by implementing **Gauss Sector Correction (GSC)** to actively correct bit-flip errors.  When a bit-flip occurs and the Gauss law is violated, instead of discarding the sample, one can identify the most likely error and determine the recovery operation to reverse it and project the state back onto the physical subspace. This is achieved using a $G_n$ stabilizer code, which — while not fully fault-tolerant — still provides a degree of partial error correction.  

</div>